## MVP Siamese LSTM Net

This is a baseline siamese LSTM net. The purpose is to build out the architecture, and see if the net can get as good as validation score as the classifiers.

In [17]:
# data manipulation
import utils
import pandas as pd
import numpy as np

# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation, Input, Add, concatenate
from keras.layers.embeddings import Embedding
from keras.utils.vis_utils import model_to_dot
from keras.callbacks import ModelCheckpoint, TensorBoard

# plotting
from IPython.display import SVG

In [2]:
X_train = utils.load('X_train')
y_train = utils.load('y_train')

## Tokenize and Encode vocabulary

1. Limit the vocab to 20,000 words.
2. Clean questions only and do not lemmatize.
3. Limit the question length to 100 tokens.

In [3]:
vocabulary_size = 20000
X_train_stack = utils.clean_questions(utils.stack_questions(X_train))

In [4]:
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(X_train_stack)

sequences = tokenizer.texts_to_sequences(X_train_stack)
data = pad_sequences(sequences, maxlen=100)

In [5]:
print(data.shape)

(606398, 100)


In [6]:
data[:,0].sum()

7212

## Embedding Matrix

1. Calculates the embedding matrix utilizing spaCy `en_core_web_lg` word vectors.
  * https://spacy.io/models/en#en_core_web_lg
  * GloVe vectors trained on Common Crawl

In [7]:
try:
    embedding_matrix = utils.load('embedding_matrix')
except:
    # create a weight matrix for words in training docs
    embedding_matrix = np.zeros((vocabulary_size, 300))
    for word, index in tokenizer.word_index.items():
    #     print(word, index, end='\r')
        if index > vocabulary_size - 1:
            break
        else:
            embedding_vector = utils.nlp(word).vector
            if embedding_vector is not None:
                embedding_matrix[index] = embedding_vector
    #     break

    utils.save(embedding_matrix, 'embedding_matrix')

## Define the batch to pass into the network

Create arrays to split the stacked data into question 1 set and question 2 set for each pair.

**Need to cleanup this cell in the next model**

In [20]:
odd_idx = [i for i in range(data.shape[0]) if i % 2 == 1]
even_idx = [i for i in range(data.shape[0]) if i % 2 == 0]

## TESTING - cleanup in next notebook 
# train_data_x1 = data[odd_idx[:10000]]
# train_data_x2 = data[even_idx[:10000]]
# train_labels = y_train[:10000]

# val_data_x1 = data[odd_idx[10000:11000]]
# val_data_x2 = data[even_idx[10000:11000]]
# val_labels = y_train[10000:11000]

print(f'Train neg class: {len(train_labels[train_labels == 0]) / len(train_labels):.2}')
print(f'Val neg class: {len(val_labels[val_labels == 0]) / len(val_labels):.2}')

Train neg class: 0.63
Val neg class: 0.61


## Build out legs of the siamese network

The architecure is the following,

0. Input - (100,) word tensor
1. Embedding Layer - outputs (300,) **not trainable**
2. LSTM - default outputs (300,)
3. Concatenate the two nets outputs (600,)
4. Dropout - 20%
5. Dense - outputs (100,), activation `tanh` -- somewhat random decision
6. Dropout - 20%
7. Dense - outputs (1,), activation `sigmoid`

In [19]:
# Creating word embedding layer
embedding_layer = Embedding(vocabulary_size, 300, input_length=100, 
                                     weights=[embedding_matrix], trainable=False)

# Creating LSTM Encoder
# Bidirectional(LSTM(self.number_lstm_units, dropout=self.rate_drop_lstm, recurrent_dropout=self.rate_drop_lstm))
lstm_layer = LSTM(300)

# Creating LSTM Encoder layer for First Sentence
sequence_1_input = Input(shape=(100,), dtype='int32')
embedded_sequences_1 = embedding_layer(sequence_1_input)
x1 = lstm_layer(embedded_sequences_1)

# Creating LSTM Encoder layer for Second Sentence
sequence_2_input = Input(shape=(100,), dtype='int32')
embedded_sequences_2 = embedding_layer(sequence_2_input)
x2 = lstm_layer(embedded_sequences_2)



In [21]:
# Merging two LSTM encodes vectors from sentences to
# pass it to dense layer applying dropout and batch normalisation
merged = concatenate([x1, x2])
# merged = BatchNormalization()(merged)
merged = Dropout(.2)(merged)
merged = Dense(100, activation='tanh')(merged)
# merged = BatchNormalization()(merged)
merged = Dropout(0.2)(merged)
preds = Dense(1, activation='sigmoid')(merged)

model = Model(inputs=[sequence_1_input, sequence_2_input], outputs=preds)
model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['acc'])
# SVG(model_to_dot(model, show_shapes=True).create(prog='dot', format='svg'))

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 100, 300)     6000000     input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 300)          721200      embedding_1[0][0]                
          

In [22]:
file_path = '../data/keras_models/mvp_{epoch:02d}-{val_loss:.2f}.hdf5'
model_checkpoint = ModelCheckpoint(filepath=file_path)


tensorboard = TensorBoard(log_dir='../data/tensorboard')

In [23]:
model.fit([data[odd_idx], data[even_idx]], y_train, validation_split=0.33,
                  epochs=10, batch_size=64, shuffle=True,
                  callbacks=[model_checkpoint, tensorboard])

Train on 203143 samples, validate on 100056 samples
Epoch 1/10
203143/203143 [==============================] - 1102s 5ms/step - loss: 0.5499 - acc: 0.7221 - val_loss: 0.5066 - val_acc: 0.7530
Epoch 2/10
203143/203143 [==============================] - 1034s 5ms/step - loss: 0.4834 - acc: 0.7684 - val_loss: 0.4723 - val_acc: 0.7756
Epoch 3/10
203143/203143 [==============================] - 1119s 6ms/step - loss: 0.4418 - acc: 0.7952 - val_loss: 0.4578 - val_acc: 0.7838
Epoch 4/10
203143/203143 [==============================] - 1255s 6ms/step - loss: 0.4591 - acc: 0.7816 - val_loss: 0.4670 - val_acc: 0.7790
Epoch 5/10
203143/203143 [==============================] - 1098s 5ms/step - loss: 0.4111 - acc: 0.8130 - val_loss: 0.4591 - val_acc: 0.7852
Epoch 6/10
 84864/203143 [===========>..................] - ETA: 8:05 - loss: 0.3549 - acc: 0.8450

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



203143/203143 [==============================] - 1171s 6ms/step - loss: 0.3303 - acc: 0.8579 - val_loss: 0.4551 - val_acc: 0.7978
Epoch 8/10
203143/203143 [==============================] - 1441s 7ms/step - loss: 0.3001 - acc: 0.8733 - val_loss: 0.4715 - val_acc: 0.7984
Epoch 9/10
203143/203143 [==============================] - 1328s 7ms/step - loss: 0.2765 - acc: 0.8849 - val_loss: 0.4898 - val_acc: 0.7939
Epoch 10/10
203143/203143 [==============================] - 1670s 8ms/step - loss: 0.3228 - acc: 0.8581 - val_loss: 0.4729 - val_acc: 0.7835


## Results

The best average validation accuracy from the classification models is 0.783667, so very similar validation accuracy.

### Next Steps
Implement one or many of the ideas below.

Future Ideas:
* Explore LSTM settings
* Dropout rates
* Adding or removing the dense layers
* Add BatchNormalization - theoritically speeds up training
  * https://arxiv.org/pdf/1502.03167.pdf
* Add EarlyStopping

Change the validation scoring AUC, since the majority class represents 63% of the data.

## Example Code

Example below is how to calculate AUC at the end of each epoch on the validation data. **Add this to the next model**.

https://gist.github.com/smly/d29d079100f8d81b905e

In [ ]:
"""
An example to check the AUC score on a validation set for each 10 epochs.
I hope it will be helpful for optimizing number of epochs.
"""

'''
# -*- coding: utf-8 -*-
import logging

from sklearn.metrics import roc_auc_score
from keras.callbacks import Callback


class IntervalEvaluation(Callback):
    def __init__(self, validation_data=(), interval=10):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict_proba(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            logging.info("interval evaluation - epoch: {:d} - score: {:.6f}".format(epoch, score))
            
'''